In [1]:
from cad import CAD
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name = "/root/share/chinese-alpaca-2-13b"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, )
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, padding_side="left", )
cad = CAD(model=model, tokenizer=tokenizer)

/opt/conda/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-05 12:18:12,490] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.58s/it]
/opt/conda/envs/LLM/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/LLM/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  wa

In [4]:
import pandas as pd

PROMPT_TEMPLATE = """[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

【已知】：【"项目名称"：{title}
{label}：{content}】
你现在是一个政企领域的专家，基于【已知】给的多个内容，请回答问题：“{question}”，一定要保证答案正确！[/INST]"""

df = pd.read_json("/root/share/scripts/output/intent-chatglm3-6b-optimize.json")
row = df.loc[0]
question = row["一般提问"]
prompt = PROMPT_TEMPLATE.format(
    title=row["title"],
    label=row["二级policy预测"],
    content=row["content"],
    question=row["一般提问"],
)

print(prompt)

[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

【已知】：【"项目名称"：2022年工业互联网标识解析二级节点建设奖励
审批流程：无。】
你现在是一个政企领域的专家，基于【已知】给的多个内容，请回答问题：“2022年工业互联网标识解析二级节点建设奖励的审批流程是什么？”，一定要保证答案正确！[/INST]


In [7]:
outputs_no_cad = cad.generate(
    texts=[prompt],
    max_new_tokens=200,
)
print(outputs_no_cad)

['根据【已知】的内容，2022年工业互联网标识解析二级节点建设奖励的审批流程是无。也就是说，没有具体的审批程序或步骤需要进行申请和审核的过程。']


In [8]:
outputs = cad.generate(
    texts=[question],
    texts_with_context=[prompt],
    max_new_tokens=200,
)

print(outputs)

['\n答：工业和信息化部会同有关部门制定《关于做好2019年-2022年全国性行业级标识解析二级节点建设工作的通知》（工信厅〔2018〕36号）。 根据该文件，省级通信管理局负责组织本地区的相关工作并向本部门提出申请；国家互联网信息办公室对符合条件的应用进行审核后报国务院办公厅批准发布公告。']
